Let's practice making a simple Kalman filter that can filter noisy sensor measurements!

We'll pretend to be a robot using some noisy sensor to sense its location. The true position of our robot will be given by $x$. The measurements, $z$, are Gaussian random values centered about the actual position $x$ with a variance given by $R$.

In [ ]:
%matplotlib notebook

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

Setup the initial parameters.

In [ ]:
#setup initial parameters
n_iter = 100
sz = (n_iter,)	#Size of array
x0 = -0.5	#Initial position value
move = True #Enable or disable random movement
u_const = 0.1

R = 0.1**2 # Estimate of measurement variance
Q = 1.0 #Process variance
T = 0.5 #control input generation variance
U = 0.3 #control input variance

#Create the arrays
x = np.zeros(sz)	# array of actual position values
u = np.zeros(sz)    # array of control inputs
z = np.zeros(sz)	# measurement array


xhat = np.zeros(sz)	# a posteri estimate of x
Vhat = np.zeros(sz)	# a posteri error estimate (variance)
#xhatminus = np.zeros(sz)	# a priori estimate of x
#Vhatminus = np.zeros(sz)	# a priori error estimate (variance)

K = np.zeros(sz)	# Kalman gain or blending factor

# initial guess
xhat[0] = 0.0
#P[0] = 1.0
Vhat[0] = R

# initial position
x[0] = x0

Recursively apply the Kalman filter algorithm

In [ ]:
def kalman_filter(mu_minus, var_minus, u, z):
    # calculate the predicted belief
    mu_hat = mu_minus + u
    var_hat = var_minus + R
    
    # calculate the Kalman gain
    K = var_minus/(var_minus + Q)
    print (K)
    # update the belief
    mu_t = mu_hat + K*(z - mu_hat)
    var_t = (1 - K) * var_hat
    
    return mu_t, var_t
    
    
for k in range(1, n_iter):
    # generate control inputs
    if move is True:
        #u[k] = np.random.randn()*T # random-walk movement
        #u[k] = u_const # constant movement
        u[k] = np.sin(np.radians(k*7))/10 # sinusoidal movement
    
    # move the robot and add in noise
    x[k] = x[k-1] + u[k] + np.random.randn()*0.1
    
    # generate a sensor measurement about x
    z[k] = np.random.randn()*R + x[k]
    
    #apply the kalman filter
    xhat[k], Vhat[k] = kalman_filter(xhat[k-1], Vhat[k-1] + Q, u[k], z[k])
    print("%1.3f, %1.3f" %(xhat[k], Vhat[k]))

fig = plt.figure()
plt.plot(z, 'k+', label='noisy measurements')
plt.plot(xhat, 'b-', label='a posteri estimate')
plt.plot(xhat + Vhat, 'g-', label = 'a posteri error estimate')
plt.plot(xhat - Vhat, 'g-')
plt.plot(x, 'r-', label='actual position')
#plt.axhline(x, color='g', label = 'truth value')
plt.legend()
plt.title('Estimate vs. iteration step', fontweight='bold')
plt.xlabel('Iteration')
plt.ylabel('Position')